In [0]:
df = spark.read.csv("/databricks-datasets/airlines/part-00000", header=True, inferSchema=True)
df_clean = df.dropna(subset=["DepDelay", "ArrDelay", "Origin", "Dest", "UniqueCarrier"])
df_clean.printSchema()



root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Car

In [0]:
df.head()

Row(Year=1987, Month=10, DayofMonth=14, DayOfWeek=3, DepTime='741', CRSDepTime=730, ArrTime='912', CRSArrTime=849, UniqueCarrier='PS', FlightNum=1451, TailNum='NA', ActualElapsedTime='91', CRSElapsedTime=79, AirTime='NA', ArrDelay=23.0, DepDelay=11.0, Origin='SAN', Dest='SFO', Distance='447', TaxiIn='NA', TaxiOut='NA', Cancelled=0, CancellationCode='NA', Diverted=0, CarrierDelay='NA', WeatherDelay='NA', NASDelay='NA', SecurityDelay='NA', LateAircraftDelay='NA', IsArrDelayed='YES', IsDepDelayed='YES')

In [0]:
from pyspark.sql.functions import col, when, avg, round

# Average Departure & Arrival Delay per Airline
df = df_clean \
    .withColumn("DepDelay", when(col("DepDelay") != "NA", col("DepDelay").cast("double"))) \
    .withColumn("ArrDelay", when(col("ArrDelay") != "NA", col("ArrDelay").cast("double")))

df.groupBy("UniqueCarrier").agg(
    round(avg("DepDelay"),2).alias("Departure Delay"),
    round(avg("ArrDelay"), 2).alias("Arrival Delay"))\
        .orderBy(avg("ArrDelay"), ascending=True).show()

+-------------+---------------+-------------+
|UniqueCarrier|Departure Delay|Arrival Delay|
+-------------+---------------+-------------+
|           CO|           4.92|         2.02|
|           AA|           4.27|         2.44|
|           WN|           7.24|          4.6|
|           UA|           5.31|         5.37|
|           TW|           5.03|         5.87|
|           EA|           6.21|         6.27|
|       PA (1)|           4.35|          7.4|
|           PI|           6.17|         7.72|
|           US|           5.72|         7.92|
|           DL|           4.66|         8.16|
|           NW|           4.87|          8.6|
|           HP|            5.5|         9.31|
|           AS|           7.01|         10.1|
|           PS|          11.06|        15.11|
+-------------+---------------+-------------+



In [0]:
# Worst Airports for Arrival Delays

df.groupBy("Dest").agg(round(avg("ArrDelay"),2).alias("Arrival_Delay"))\
    .orderBy("Arrival_Delay", ascending=False)\
        .show()


+----+-------------+
|Dest|Arrival_Delay|
+----+-------------+
| TVL|        25.15|
| RDD|        20.14|
| ACV|        18.52|
| YKM|        18.23|
| ROR|        17.04|
| CDV|         17.0|
| WRG|        16.43|
| SFO|        16.24|
| YAK|        15.72|
| LMT|        15.42|
| PSG|        14.73|
| SIT|         14.7|
| PIE|        14.29|
| BLI|         14.1|
| BET|         13.2|
| STT|        13.04|
| MRY|        12.77|
| SNA|        12.27|
| AVL|        12.17|
| LYH|        12.15|
+----+-------------+
only showing top 20 rows


In [0]:
#  Monthly Delay Patterns

df.groupBy("Month") \
    .agg(round(avg("ArrDelay"),2).alias("Arrival_Delay"), 
        round(avg("DepDelay"),2).alias("Departure_Delay")) \
        .orderBy("Month") \
            .show()

+-----+-------------+---------------+
|Month|Arrival_Delay|Departure_Delay|
+-----+-------------+---------------+
|   10|          6.0|           5.06|
|   11|         8.05|           6.91|
+-----+-------------+---------------+



In [0]:
display(df.groupBy("Month").agg(avg("ArrDelay").alias("AvgArrDelay")))


Month,AvgArrDelay
10,6.004721435316336
11,8.049828355294263


In [0]:
df.createOrReplaceTempView("Flights")

In [0]:
%sql
SELECT Month, AVG(ArrDelay) AS Arrival_Delay
FROM Flights
GROUP BY Month
ORDER BY Month


Month,Arrival_Delay
10,6.004721435316336
11,8.049828355294263
